# **Clase 2: Spring Data JPA – Entidades, Repositorios y Relaciones**
---

## **Objetivos de Aprendizaje**

Al finalizar la clase, los estudiantes serán capaces de:

* Mapear clases Java a tablas en base de datos con JPA de manera profesional.
* Comprender e implementar relaciones entre entidades con sus atributos (cascade, fetch, mappedBy).
* Construir interfaces `Repository` y utilizar `JpaRepository`/`CrudRepository` para operaciones CRUD limpias.
* Validar entidades y asegurar consistencia entre modelo y base de datos.
* Aplicar buenas prácticas y normas internacionales como **Java Persistence API (JPA) 2.2**, **ISO/IEC 25010**, y guías de diseño limpio.

---



## **Contenidos y Estructura de la Clase**

### **Bloque 1: Introducción a JPA y Anotaciones (@Entity, @Id, etc.)**

### ¿Qué es JPA?

**JPA (Java Persistence API)** es una especificación de Java que permite el mapeo entre **objetos de dominio (clases Java)** y **tablas relacionales en una base de datos**. Su objetivo es facilitar la persistencia de datos, ocultando el uso directo de SQL.

> ⚠️ JPA no es una implementación: solo define un conjunto de reglas e interfaces.

### ¿Qué es Hibernate?

**Hibernate** es una implementación concreta de JPA. Spring Boot utiliza Hibernate por defecto cuando se incluye Spring Data JPA.

---

## **Anotaciones básicas de JPA**

### 1. `@Entity`

* Indica que una clase es una entidad y debe ser persistida en una tabla de base de datos.
* Debe tener un constructor vacío y un atributo `@Id`.



In [ ]:
@Entity
public class Cliente {
    //...
}


---

### 2. `@Table(name = "clientes")`

* (Opcional) Especifica el nombre exacto de la tabla en la base de datos.
* Si no se indica, se toma el nombre de la clase como nombre de tabla.



In [ ]:
@entity
@Table(name = "clientes")
public class Cliente {}

---

### 3. `@Id`

* Marca el atributo que actúa como clave primaria.


In [ ]:
@entity
@Table(name = "clientes")
public class Cliente {

    @Id
    private Long id;
}


---

### 4. `@GeneratedValue`

* Indica que el valor del campo será generado automáticamente.



In [ ]:
@entity
@Table(name = "clientes")
public class Cliente {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

}

* **Strategies más comunes:**

  * `AUTO`: El proveedor decide la mejor estrategia.
  * `IDENTITY`: Usa autoincremento.
  * `SEQUENCE`: Usa secuencias (más común en Oracle/PostgreSQL).

---


### 5. `@Column`

* Personaliza las propiedades de las columnas: nombre, tamaño, si permite nulos, etc.



In [ ]:
    @Column(nullable = false)
    private String nombre;

    @Column(nullable = false, unique = true)
    private String email;
    
    private String telefono;

---

## **Buenas prácticas internacionales (ISO/IEC 25010 + OWASP Secure Coding)**

| Práctica                                                          | Justificación                                       |
| ----------------------------------------------------------------- | --------------------------------------------------- |
| Usar tipos envolventes (`Long`, `Integer`) en lugar de primitivos | Permite representar valores nulos correctamente     |
| Nombrar explícitamente columnas y tablas                          | Mejora la mantenibilidad y el desacoplamiento       |
| Añadir restricciones (`nullable = false`, `unique = true`)        | Refuerza la integridad de datos                     |
| Validar entradas antes de persistir                               | Evita errores de negocio y de seguridad (inyección) |

---


## **Ejercicio práctico empresarial**

> **Caso real:** Una empresa desea registrar los datos de sus clientes, asegurándose de que cada uno tenga un correo único y un número de teléfono opcional.

### Crear la clase `Cliente`



### ¿Dónde se guarda esta clase `Cliente.java`?

Se ubica dentro del **paquete de dominio o entidades**, que usualmente se estructura así:

```
src/
└── main/
    └── java/
        └── com/
            └── empresa/
                └── app/
                    └── entity/      ←  Aquí va la clase Cliente
                        └── Cliente.java
```


In [ ]:
package com.empresa.app.entity;

import javax.persistence.*;
import javax.validation.constraints.*;
import com.fasterxml.jackson.annotation.JsonManagedReference;

import java.util.List;

@Entity
@Table(name = "clientes")
public class Cliente {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    @Column(nullable = false)
    @NotNull
    @Size(min = 2)
    private String nombre;

    @Column(nullable = false, unique = true)
    @Email
    @NotNull
    private String email;

    private String telefono;

    @OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL, fetch = FetchType.LAZY)
    @JsonManagedReference
    private List<Pedido> pedidos;

    public Cliente() {
    }

    public Cliente(String nombre, String email, String telefono) {
        this.nombre = nombre;
        this.email = email;
        this.telefono = telefono;
    }

    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public String getNombre() {
        return nombre;
    }

    public void setNombre(String nombre) {
        this.nombre = nombre;
    }

    public String getEmail() {
        return email;
    }

    public void setEmail(String email) {
        this.email = email;
    }

    public String getTelefono() {
        return telefono;
    }

    public void setTelefono(String telefono) {
        this.telefono = telefono;
    }

    public List<Pedido> getPedidos() {
        return pedidos;
    }

    public void setPedidos(List<Pedido> pedidos) {
        this.pedidos = pedidos;
    }

}


### ¿Qué representa esta clase `Cliente.java`?

* Es una **representación de una tabla en la base de datos** (`clientes`), usando anotaciones de **JPA** como `@Entity` y `@Table`.
* Pertenece a la **capa de persistencia del dominio**, pero **no contiene lógica de negocio**, solo estructura de datos y anotaciones de mapeo.
* Cumple el patrón de **Entidad Anémica** (solo estado, sin comportamiento complejo), que es adecuado para sistemas CRUD con Spring Data JPA.

---


## **Bloque 2: Creación de Repositorios con JpaRepository y CrudRepository**

### ¿Qué es un Repositorio en Spring Data JPA?

Es una **abstracción del patrón Repository**, que encapsula el acceso a datos y permite interactuar con la base de datos sin escribir SQL explícito. Spring Data JPA proporciona interfaces genéricas que gestionan automáticamente operaciones CRUD básicas y avanzadas.

---

## **Interfaces disponibles**

### 1. `CrudRepository<T, ID>`

```java
public interface ClienteRepository extends CrudRepository<Cliente, Long> { }
```

### ¿Qué ofrece?

Proporciona las **operaciones básicas CRUD**:

* `save(S entity)`
* `findById(ID id)`
* `findAll()`
* `deleteById(ID id)`
* `count()`

### ¿Cuándo usarlo?

* Cuando **solo necesitas CRUD básico** y deseas mantener la interfaz lo más **simple y ligera** posible.
* Ideal para **microservicios simples**, **repositorios pequeños** o cuando quieres controlar más tu implementación.


---

### 2. `JpaRepository<T, ID>`

```java
public interface ClienteRepository extends JpaRepository<Cliente, Long> { }
```

### ¿Qué ofrece además de lo básico?

`JpaRepository` **extiende a CrudRepository**, pero añade:

* Métodos más **poderosos y sofisticados**:

  * `findAll(Sort sort)`
  * `findAll(Pageable pageable)` → **paginación**
  * `flush()`, `saveAndFlush()`
* Soporte para **Batch processing** (`saveAll()`)
* **Mejor manejo de colecciones** y opciones avanzadas de persistencia.

### ¿Cuándo usarlo?

* Cuando necesitas:

  * **Paginación** y ordenamiento.
  * Consultas avanzadas.
  * Operaciones en **lotes**.
  * Más flexibilidad y eficiencia.
* Ideal para **aplicaciones medianas o grandes**, con **múltiples entidades y relaciones**.


## ¿Cuál usar entonces?

| Criterio                      | Usa `CrudRepository` | Usa `JpaRepository`                       |
| ----------------------------- | -------------------- | ----------------------------------------- |
| CRUD básico                   | ✅                    | ✅                                         |
| Paginación y ordenamiento     | ❌                    | ✅ `findAll(Pageable pageable)`            |
| Operaciones por lotes         | ❌                    | ✅ `saveAll()`                             |
| Acceso directo a JPA features | ❌                    | ✅ `flush()`, `getOne()`, `saveAndFlush()` |
| Aplicaciones simples          | ✅                    | ❌                                         |
| Aplicaciones complejas        | ❌                    | ✅                                         |

---



### **¿Cómo funciona internamente?**

Spring Data JPA implementa automáticamente estas interfaces mediante **proxies dinámicos** usando **Java Reflection y AOP (Programación Orientada a Aspectos)**.

---


## *Creación de métodos personalizados (Derived Queries)**

Spring permite crear consultas automáticamente **a partir del nombre del método**, aplicando un patrón inteligente.

### Ejemplos:



In [ ]:
// Buscar por nombre exacto
List<Cliente> findByNombre(String nombre);

// Buscar clientes cuyo email contenga un valor
List<Cliente> findByEmailContaining(String texto);

// Buscar por nombre ignorando mayúsculas/minúsculas
List<Cliente> findByNombreIgnoreCase(String nombre);

// Buscar por nombre o email
List<Cliente> findByNombreOrEmail(String nombre, String email);

// Buscar los primeros 5 clientes ordenados por ID
List<Cliente> findTop5ByOrderByIdAsc();


> Normas: Estos métodos cumplen con el principio de **abstracción y reutilización**, evitando acoplar la lógica de acceso a datos al código del negocio (ISO/IEC 25010 – mantenibilidad).

---


## **Ejercicio práctico empresarial**

> **Caso real:** El equipo de soporte necesita un sistema para buscar clientes por nombre y por parte del correo electrónico.

### Crear el repositorio `ClienteRepository`



### ¿Dónde se guarda la clase `ClienteRepository.java`?

La clase `ClienteRepository.java`, que es una **interfaz que extiende `JpaRepository` o `CrudRepository`**,  que se guarda en un paquete **específico para los repositorios**.

---

### 📂 Ubicación correcta

```java
src/
└── main/
    └── java/
        └── com/
            └── empresa/
                └── app/
                    ├── entity/        ← Aquí va Cliente.java
                    │    └── Cliente.java
                    │
                    └── repository/     ← Aquí va ClienteRepository.java
                         └── ClienteRepository.java
```



In [ ]:
package com.empresa.app.repository;

import com.empresa.app.entity.Cliente;
import org.springframework.data.jpa.repository.JpaRepository;
import java.util.List;

public interface ClienteRepository extends JpaRepository<Cliente, Long> {
    List<Cliente> findByNombre(String nombre);

    List<Cliente> findByEmailContaining(String email);
}


---

## **Inyección de dependencias (Dependency Injection)**

Spring permite inyectar los repositorios directamente en los servicios usando la anotación `@Autowired`, siguiendo el principio **D de SOLID: Inversión de dependencias**.

### Crear el servicio `ClienteService`


### ¿Dónde se guarda la clase `ClienteServices.java`?

La clase `ClienteServices.java`, o más comúnmente nombrada como `ClienteService`, representa la **capa de lógica de negocio** en una aplicación Spring Boot. Esta clase se encarga de implementar la lógica empresarial utilizando los repositorios y se anota típicamente con `@Service`.

---

### 📂 Ubicación correcta

```java
src/
└── main/
    └── java/
        └── com/
            └── empresa/
                └── app/
                    ├── entity/        ← Aquí va Cliente.java
                    │    └── Cliente.java
                    │
                    ├── repository/     ← Aquí va ClienteRepository.java
                    │    └── ClienteRepository.java
                    │
                    └── service/        ← Aquí va ClienteService.java
                         └── ClienteService.java
```



In [ ]:
package com.empresa.app.service;

import com.empresa.app.entity.Cliente;
import com.empresa.app.repositorios.ClienteRepository;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.stereotype.Service;

import java.util.List;

@Service
public class ClienteService {

    @Autowired
    private ClienteRepository clienteRepository;

    public List<Cliente> obtenerTodos() {
        return clienteRepository.findAll();
    }

    public Cliente guardar(Cliente cliente) {
        return clienteRepository.save(cliente);
    }

    public List<Cliente> buscarPorNombre(String nombre) {
        return clienteRepository.findByNombre(nombre);
    }

    public List<Cliente> buscarPorEmail(String fragmento) {
        return clienteRepository.findByEmailContaining(fragmento);
    }

    public void eliminarPorId(Long id) {
        clienteRepository.deleteById(id);
    }
}


## **Bloque 3: Relaciones entre Entidades (JPA Relationships)**

### **1. Teoría: Relaciones en JPA**

#### `@OneToOne`

Una relación donde **una entidad está asociada con una única instancia de otra entidad**.



In [ ]:
@OneToOne
@JoinColumn(name = "usuario_id")
private Usuario usuario;


Se utiliza cuando los datos están estrechamente acoplados, por ejemplo:
**Cliente** ↔ **DirecciónFiscal**

---

#### `@OneToMany` 

Relación clásica padre-hijo:
Un **Cliente** puede tener muchos **Pedidos**



In [ ]:
// Cliente.java
@OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL)
private List<Pedido> pedidos;


#### `@ManyToOne` 

Relación clásica hijo-padre:
Un **Pedido** pertenece a un solo **Cliente**.

In [ ]:
// Pedido.java
@ManyToOne
@JoinColumn(name = "cliente_id")
private Cliente cliente;

---

#### `@ManyToMany`

Cuando **muchos registros de una entidad se relacionan con muchos registros de otra**.
Ejemplo: Un pedido tiene muchos productos y un producto puede estar en varios pedidos.



In [ ]:
// Pedido.java
@ManyToMany
@JoinTable(
    name = "pedido_producto",
    joinColumns = @JoinColumn(name = "pedido_id"),
    inverseJoinColumns = @JoinColumn(name = "producto_id")
)
private List<Producto> productos;



#### `@ManyToMany`

In [ ]:
// Producto.java
@ManyToMany(mappedBy = "productos")
private List<Pedido> pedidos;


---

### **2. Atributos avanzados en relaciones**

| Atributo   | Descripción                                                                      |
| ---------- | -------------------------------------------------------------------------------- |
| `cascade`  | Define qué operaciones (persist, remove, etc.) se propagan a entidades hijas.    |
| `fetch`    | Estrategia de carga: `EAGER` (carga inmediata), `LAZY` (cuando se necesita).     |
| `mappedBy` | Se usa en el lado no propietario de la relación para evitar duplicidad de mapeo. |


#### Ejemplo:

In [ ]:
// Cliente.java
@OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL, fetch = FetchType.LAZY)
private List<Pedido> pedidos;


---

### **3. Actividad práctica guiada (30 min)**

Vamos a crear las siguientes relaciones reales en un sistema de ventas:

#### Entidades:

* **Cliente**
* **Pedido**
* **Producto**

#### 🔗 Relaciones:

* Un **Cliente** tiene muchos **Pedidos** (`@OneToMany`)
* Un **Pedido** tiene muchos **Productos** (`@ManyToMany`)

---

### Código base de las entidades

#### `entities/Cliente.java` (ya implementada previamente)



In [ ]:
// Cliente.java
@OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL, fetch = FetchType.LAZY)
private List<Pedido> pedidos;


---

#### `entities/Pedido.java`



In [ ]:
package com.empresa.app.entity;

import java.time.LocalDate;
import java.util.List;

import javax.persistence.*;

import com.fasterxml.jackson.annotation.JsonBackReference;

@Entity
@Table(name = "pedidos")
public class Pedido {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    private LocalDate fecha = LocalDate.now();

    @ManyToOne
    @JoinColumn(name = "cliente_id")
    @JsonBackReference
    private Cliente cliente;

    @ManyToMany
    @JoinTable(name = "pedido_producto", joinColumns = @JoinColumn(name = "pedido_id"), inverseJoinColumns = @JoinColumn(name = "producto_id"))
    private List<Producto> productos;

    public Pedido() {
    }

    public Pedido(LocalDate fecha, Cliente cliente) {
        this.fecha = fecha;
        this.cliente = cliente;
    }

    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public LocalDate getFecha() {
        return fecha;
    }

    public void setFecha(LocalDate fecha) {
        this.fecha = fecha;
    }

    public Cliente getCliente() {
        return cliente;
    }

    public void setCliente(Cliente cliente) {
        this.cliente = cliente;
    }

    public List<Producto> getProductos() {
        return productos;
    }

    public void setProductos(List<Producto> productos) {
        this.productos = productos;
    }

}


---

#### `entities/Producto.java`



In [ ]:
package com.empresa.app.entity;

import javax.persistence.*;
import javax.validation.constraints.*;
import java.util.List;

@Entity
@Table(name = "productos")
public class Producto {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    @Column(nullable = false)
    @NotNull
    @Size(min = 3)
    private String nombre;

    @Column(nullable = false)
    @Min(0)
    private Double precio;

    @Column(nullable = false)
    @Min(1)
    private Integer stock;

    @ManyToMany(mappedBy = "productos")
    private List<Pedido> pedidos;

    public Producto() {
    }

    public Producto(String nombre, Double precio, Integer stock) {
        this.nombre = nombre;
        this.precio = precio;
        this.stock = stock;
    }

    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public String getNombre() {
        return nombre;
    }

    public void setNombre(String nombre) {
        this.nombre = nombre;
    }

    public Double getPrecio() {
        return precio;
    }

    public void setPrecio(Double precio) {
        this.precio = precio;
    }

    public Integer getStock() {
        return stock;
    }

    public void setStock(Integer stock) {
        this.stock = stock;
    }

    public List<Pedido> getPedidos() {
        return pedidos;
    }

    public void setPedidos(List<Pedido> pedidos) {
        this.pedidos = pedidos;
    }
}


### Resultado esperado

Al ejecutar la aplicación con estas relaciones:

* Al guardar un `Pedido` con varios `Productos`, JPA persistirá automáticamente la relación en la tabla intermedia `pedido_producto`.
* Al recuperar un `Cliente`, se puede acceder a todos sus pedidos y sus productos relacionados.
* Se aplican `fetch = LAZY` y `cascade = ALL` solo donde tiene sentido, evitando cargas innecesarias.

---


## **Actualizar empresa/App.java**


Se actualiza la clase principal App.java para que sea una clase de arranque de Spring Boot, agregando la anotación @SpringBootApplication y configurando el método main para iniciar la aplicación.

In [ ]:
package com.empresa;

import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;

@SpringBootApplication
public class App {
    public static void main(String[] args) {
        SpringApplication.run(App.class, args);
    }
}


## **Actualizar pom.xml**

Se agregan las dependencias de Spring Boot y Spring Data JPA al pom.xml para integrar Spring Boot en el proyecto

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<project xmlns="http://maven.apache.org/POM/4.0.0"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <parent>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-parent</artifactId>
    <version>2.7.0</version>
    <relativePath> </relativePath>
  </parent>

  <groupId>com.empresa</groupId>
  <artifactId>empresa</artifactId>
  <version>1.0-SNAPSHOT</version>
  <name>empresa</name>

  <properties>
    <java.version>17</java.version>
  </properties>

  <dependencies>
    <!-- Spring Boot -->
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-data-jpa</artifactId>
    </dependency>

    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-web</artifactId>
    </dependency>

    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-validation</artifactId>
    </dependency>

    <!-- JDBC Driver -->
    <dependency>
      <groupId>org.postgresql</groupId>
      <artifactId>postgresql</artifactId>
      <version>42.2.27</version>
    </dependency>

    <!-- JUnit -->
    <dependency>
      <groupId>junit</groupId>
      <artifactId>junit</artifactId>
      <version>4.13.2</version>
      <scope>test</scope>
    </dependency>
  </dependencies>

  <build>
    <plugins>
      <plugin>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-maven-plugin</artifactId>
      </plugin>

      <plugin>
        <groupId>org.apache.maven.plugins</groupId>
        <artifactId>maven-compiler-plugin</artifactId>
        <version>3.8.1</version>
        <configuration>
          <source>${java.version}</source>
          <target>${java.version}</target>
          <encoding>UTF-8</encoding>
        </configuration>
      </plugin>

    </plugins>
  </build>
</project>

---

## **Bloque 4: Proyecto en Clase – CRUD + Relaciones**

### **Modelo del dominio**

#### Entidades involucradas:

* **Cliente**
* **Pedido**
* **Producto**

#### Relaciones:

* `Cliente (1) ↔ (N) Pedido` (OneToMany)
* `Pedido (N) ↔ (M) Producto` (ManyToMany)

---

### Buenas prácticas aplicadas

**Arquitectura en capas clara:**

* `Controller → Service → Repository → Entity`

**Validaciones de entrada:**

* `@NotNull`, `@Email`, `@Size`, `@Min` en atributos clave de entidades.
* Validación automática con `@Valid` en controladores.

**Uso de `FetchType.LAZY` donde es posible:**

* Evita sobrecarga innecesaria al cargar relaciones grandes.

**Mapeo de relaciones eficiente:**

* Uso de `@JsonManagedReference` y `@JsonBackReference` para evitar ciclos en respuestas JSON.


**Manejo de errores y respuestas:**

* Uso de `ResponseEntity` para control de códigos de estado HTTP.
* Se puede extender con `@ControllerAdvice` para capturar errores globales.


---

### **Componentes a implementar**

#### 1. **Entidades (paquete `com.empresa.app.entity`)**

`entity/Cliente.java`

In [ ]:
@Entity
public class Cliente {
    @Id @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

@Column(nullable = false)
    @NotNull @Size(min = 2)
    private String nombre;

@Column(nullable = false, unique = true)
    @Email @NotNull
    private String email;

    private String telefono;

    @OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL, fetch = FetchType.LAZY)
    @JsonManagedReference
    private List<Pedido> pedidos;
}


`entity/Pedido.java`

In [ ]:
@Entity
@Table(name = "pedidos")
public class Pedido {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    private LocalDate fecha = LocalDate.now();

    @ManyToOne
    @JoinColumn(name = "cliente_id")
    @JsonBackReference
    private Cliente cliente;

    @ManyToMany
    @JoinTable(name = "pedido_producto", joinColumns = @JoinColumn(name = "pedido_id"), inverseJoinColumns = @JoinColumn(name = "producto_id"))
    private List<Producto> productos;

...
}

`entity/Producto`

In [ ]:
@Entity
@Table(name = "productos")
public class Producto {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    @Column(nullable = false)
    @NotNull
    @Size(min = 3)
    private String nombre;

    @Column(nullable = false)
    @Min(0)
    private Double precio;

    @Column(nullable = false)
    @Min(1)
    private Integer stock;

    @ManyToMany(mappedBy = "productos")
    private List<Pedido> pedidos;

...
}

---

### Repositorio `ProductoRepository`


In [ ]:
package com.empresa.app.repository;

import com.empresa.app.entity.Producto;
import org.springframework.data.jpa.repository.JpaRepository;

import java.util.List;

public interface ProductoRepository extends JpaRepository<Producto, Long> {

    // Buscar por nombre exacto
    List<Producto> findByNombre(String nombre);

    // Buscar productos cuyo nombre contenga una cadena (ej: búsqueda parcial)
    List<Producto> findByNombreContainingIgnoreCase(String nombre);

    // Buscar productos por rango de precio
    List<Producto> findByPrecioBetween(Double min, Double max);
}


### Repositorio `PedidoRepository`

In [ ]:
package com.empresa.app.repository;

import com.empresa.app.entity.Pedido;
import com.empresa.app.entity.Cliente;
import org.springframework.data.jpa.repository.JpaRepository;

import java.util.List;

public interface PedidoRepository extends JpaRepository<Pedido, Long> {

    // Buscar pedidos por cliente
    List<Pedido> findByCliente(Cliente cliente);
}


---

#### 3. **Servicios (paquete `com.empresa.app.service`)**


`service/ClienteService.java`


In [ ]:
package com.empresa.app.service;

import com.empresa.app.entity.Cliente;
import com.empresa.app.repository.ClienteRepository;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.stereotype.Service;

import java.util.List;

@Service
public class ClienteService {

    @Autowired
    private ClienteRepository clienteRepository;

    public List<Cliente> obtenerTodos() {
        return clienteRepository.findAll();
    }

    public Cliente guardar(Cliente cliente) {
        return clienteRepository.save(cliente);
    }

    public List<Cliente> buscarPorNombre(String nombre) {
        return clienteRepository.findByNombre(nombre);
    }

    public List<Cliente> buscarPorEmail(String fragmento) {
        return clienteRepository.findByEmailContaining(fragmento);
    }

    public void eliminarPorId(Long id) {
        clienteRepository.deleteById(id);
    }

    public Cliente obtenerPorId(Long id) {
        return clienteRepository.findById(id).orElse(null);
    }
}


`service/PedidoService.java`

In [ ]:
package com.empresa.app.service;

import com.empresa.app.entity.Pedido;
import com.empresa.app.entity.Cliente;
import com.empresa.app.repository.PedidoRepository;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.stereotype.Service;

import java.util.List;

@Service
public class PedidoService {
    @Autowired
    private PedidoRepository pedidoRepository;

    public List<Pedido> obtenerTodos() {
        return pedidoRepository.findAll();
    }

    public Pedido guardar(Pedido pedido) {
        return pedidoRepository.save(pedido);
    }

    public List<Pedido> obtenerPedidosPorCliente(Cliente cliente) {
        return pedidoRepository.findByCliente(cliente);
    }

    public void eliminarPorId(Long id) {
        pedidoRepository.deleteById(id);
    }
}


`service/ProductoService.java`

In [ ]:
package com.empresa.app.service;

import com.empresa.app.entity.Producto;
import com.empresa.app.repository.ProductoRepository;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.stereotype.Service;

import java.util.List;

@Service
public class ProductoService {
    @Autowired
    private ProductoRepository productoRepository;

    public List<Producto> obtenerTodos() {
        return productoRepository.findAll();
    }

    public Producto guardar(Producto producto) {
        return productoRepository.save(producto);
    }

    public List<Producto> buscarPorNombre(String nombre) {
        return productoRepository.findByNombre(nombre);
    }

    public List<Producto> buscarPorNombreParcial(String nombre) {
        return productoRepository.findByNombreContainingIgnoreCase(nombre);
    }

    public List<Producto> buscarPorRangoPrecio(Double min, Double max) {
        return productoRepository.findByPrecioBetween(min, max);
    }

    public void eliminarPorId(Long id) {
        productoRepository.deleteById(id);
    }
}


---

#### 4. **Controladores REST (paquete `com.empresa.app.controller`)**


`controller/ClienteController.java`


In [ ]:
package com.empresa.app.controller;

import com.empresa.app.entity.Cliente;
import com.empresa.app.service.ClienteService;
import javax.validation.Valid;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

import java.util.List;

@RestController
@RequestMapping("/clientes")
public class ClienteController {

    private final ClienteService clienteService;

    public ClienteController(ClienteService clienteService) {
        this.clienteService = clienteService;
    }

    @GetMapping
    public ResponseEntity<List<Cliente>> obtenerTodos() {
        List<Cliente> clientes = clienteService.obtenerTodos();
        if (clientes.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(clientes);
    }

    @PostMapping
    public ResponseEntity<Cliente> crear(@RequestBody @Valid Cliente cliente) {
        Cliente creado = clienteService.guardar(cliente);
        return ResponseEntity.status(HttpStatus.CREATED).body(creado);
    }

    @GetMapping("/nombre/{nombre}")
    public ResponseEntity<List<Cliente>> buscarPorNombre(@PathVariable String nombre) {
        List<Cliente> clientes = clienteService.buscarPorNombre(nombre);
        if (clientes.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(clientes);
    }

    @GetMapping("/email/{email}")
    public ResponseEntity<List<Cliente>> buscarPorEmail(@PathVariable String email) {
        List<Cliente> clientes = clienteService.buscarPorEmail(email);
        if (clientes.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(clientes);
    }

    @DeleteMapping("/{id}")
    public ResponseEntity<Void> eliminar(@PathVariable Long id) {
        clienteService.eliminarPorId(id);
        return ResponseEntity.noContent().build();
    }

}


`controller/PedidoController.java`

In [ ]:
package com.empresa.app.controller;

import com.empresa.app.entity.Cliente;
import com.empresa.app.entity.Pedido;
import com.empresa.app.service.ClienteService;
import com.empresa.app.service.PedidoService;
import javax.validation.Valid;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

import java.util.List;

@RestController
@RequestMapping("/pedidos")
public class PedidoController {

    private final PedidoService pedidoService;
    private final ClienteService clienteService;

    public PedidoController(PedidoService pedidoService, ClienteService clienteService) {
        this.pedidoService = pedidoService;
        this.clienteService = clienteService;
    }

    @GetMapping
    public ResponseEntity<List<Pedido>> obtenerTodos() {
        List<Pedido> pedidos = pedidoService.obtenerTodos();
        if (pedidos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(pedidos);
    }

    @PostMapping
    public ResponseEntity<Pedido> crear(@RequestBody @Valid Pedido pedido) {
        Pedido creado = pedidoService.guardar(pedido);
        return ResponseEntity.status(HttpStatus.CREATED).body(creado);
    }

    @GetMapping("/cliente/{clienteId}")
    public ResponseEntity<List<Pedido>> listarPorCliente(@PathVariable Long clienteId) {
        Cliente cliente = clienteService.obtenerPorId(clienteId);
        if (cliente == null) {
            return ResponseEntity.notFound().build();
        }
        List<Pedido> pedidos = pedidoService.obtenerPedidosPorCliente(cliente);
        if (pedidos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(pedidos);
    }

    @DeleteMapping("/{id}")
    public ResponseEntity<Void> eliminar(@PathVariable Long id) {
        pedidoService.eliminarPorId(id);
        return ResponseEntity.noContent().build();
    }

}


`controller/ProductoController.java`

In [ ]:
package com.empresa.app.controller;

import com.empresa.app.entity.Producto;
import com.empresa.app.service.ProductoService;
import javax.validation.Valid;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

import java.util.List;

@RestController
@RequestMapping("/productos")
public class ProductoController {

    private final ProductoService productoService;

    public ProductoController(ProductoService productoService) {
        this.productoService = productoService;
    }

    @GetMapping
    public ResponseEntity<List<Producto>> obtenerTodos() {
        List<Producto> productos = productoService.obtenerTodos();
        if (productos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(productos);
    }

    @PostMapping
    public ResponseEntity<Producto> crear(@RequestBody @Valid Producto producto) {
        Producto creado = productoService.guardar(producto);
        return ResponseEntity.status(HttpStatus.CREATED).body(creado);
    }

    @GetMapping("/nombre/{nombre}")
    public ResponseEntity<List<Producto>> buscarPorNombre(@PathVariable String nombre) {
        List<Producto> productos = productoService.buscarPorNombre(nombre);
        if (productos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(productos);
    }

    @GetMapping("/nombre/parcial/{nombre}")
    public ResponseEntity<List<Producto>> buscarPorNombreParcial(@PathVariable String nombre) {
        List<Producto> productos = productoService.buscarPorNombreParcial(nombre);
        if (productos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(productos);
    }

    @GetMapping("/precio")
    public ResponseEntity<List<Producto>> buscarPorRangoPrecio(
            @RequestParam Double min,
            @RequestParam Double max) {
        List<Producto> productos = productoService.buscarPorRangoPrecio(min, max);
        if (productos.isEmpty()) {
            return ResponseEntity.noContent().build();
        }
        return ResponseEntity.ok(productos);
    }

    @DeleteMapping("/{id}")
    public ResponseEntity<Void> eliminar(@PathVariable Long id) {
        productoService.eliminarPorId(id);
        return ResponseEntity.noContent().build();
    }

}


## **Bloque 5: Configuración, Ejecución y Pruebas del Proyecto**

## **Fundamento teórico**

**¿Por qué PostgreSQL?**
PostgreSQL es una base de datos relacional de código abierto, altamente compatible con JPA y ampliamente utilizada en entornos empresariales.

Spring Boot **no crea bases de datos desde cero automáticamente en PostgreSQL** como lo haría en H2. **Pero sí puede conectarse a una base existente** y generar todas las tablas y relaciones a partir de las entidades JPA si:

1. El usuario de PostgreSQL tiene permisos suficientes (como `CREATEDB` o acceso a una base preexistente).
2. Configuras correctamente el esquema y la URL de conexión.
3. Si la base no existe, puedes automatizar su creación con un script SQL que Spring Boot ejecute al iniciar.



**Arquitectura del flujo de datos en Spring Boot:**

```
[Entity] ←→ [JPA Repository] ←→ [Service] ←→ [Controller] ←→ [Postman]
                                         ↓
                                    [Base de datos PostgreSQL]
```

## Estrategias disponibles

| Estrategia                                                         | Descripción                                  | Nivel de automatización |
| ------------------------------------------------------------------ | -------------------------------------------- | ----------------------- |
| ✅ Crear la base manualmente 1 vez y dejar que Spring genere tablas | Más común y segura                           | Media                   |
| ⚠️ Automatizar creación de la base mediante scripts SQL (opcional) | Avanzado, pero necesita permisos elevados    | Alta                    |
| 🚫 Esperar que Spring cree la base automáticamente                 | No es compatible con PostgreSQL directamente | No funciona             |



## ¿Qué haremos?

Usaremos la opción **intermedia más estable**:

1. Crear un esquema general llamado `postgres` (si no existe).
2. Dejar que Spring Boot **cree automáticamente todas las tablas y relaciones**.
3. Usar `spring.jpa.hibernate.ddl-auto=update` para permitir la creación y actualización de estructuras de tabla.




Spring Boot usa el archivo `application.properties` o `application.yml` para establecer las conexiones de base de datos y controlar el comportamiento de JPA (como la creación automática de tablas).

---

## **Configuración Práctica Paso a Paso**

### Paso 1: Agregar PostgreSQL en `pom.xml`



In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<project xmlns="http://maven.apache.org/POM/4.0.0"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <parent>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-parent</artifactId>
    <version>2.7.0</version>
    <relativePath> </relativePath>
  </parent>

  <groupId>com.empresa</groupId>
  <artifactId>empresa</artifactId>
  <version>1.0-SNAPSHOT</version>
  <name>empresa</name>

  <properties>
    <java.version>17</java.version>
  </properties>

  <dependencies>
    <!-- Spring Boot -->
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-data-jpa</artifactId>
    </dependency>

    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-web</artifactId>
    </dependency>

    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-validation</artifactId>
    </dependency>

    <!-- JDBC Driver -->
    <dependency>
      <groupId>org.postgresql</groupId>
      <artifactId>postgresql</artifactId>
      <version>42.2.27</version>
    </dependency>

    <!-- JUnit -->
    <dependency>
      <groupId>junit</groupId>
      <artifactId>junit</artifactId>
      <version>4.13.2</version>
      <scope>test</scope>
    </dependency>
  </dependencies>

  <build>
    <plugins>
      <plugin>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-maven-plugin</artifactId>
      </plugin>

      <plugin>
        <groupId>org.apache.maven.plugins</groupId>
        <artifactId>maven-compiler-plugin</artifactId>
        <version>3.8.1</version>
        <configuration>
          <source>${java.version}</source>
          <target>${java.version}</target>
          <encoding>UTF-8</encoding>
        </configuration>
      </plugin>

    </plugins>
  </build>
</project>

### Paso 2: Configuración de `main/resources/application.properties` para crear tablas automáticamente



In [ ]:
# Configuración de conexión
spring.datasource.url=jdbc:postgresql://localhost:5432/mi_base_de_datos
spring.datasource.username=postgres
spring.datasource.password=1126254560


# Driver y plataforma
spring.datasource.driver-class-name=org.postgresql.Driver
spring.jpa.database-platform=org.hibernate.dialect.PostgreSQLDialect

# Auto-crear tablas (solo para desarrollo)
spring.jpa.hibernate.ddl-auto=update

# Mostrar SQL en consola (útil para depuración)
spring.jpa.show-sql=true
spring.jpa.properties.hibernate.format_sql=true

# Zona horaria (opcional)
spring.jpa.properties.hibernate.jdbc.time_zone=UTC

# Deshabilitar la inicialización del esquema ya que usamos JPA
spring.sql.init.mode=never



*Recomendación:* Usa `spring.jpa.hibernate.ddl-auto=validate` en producción, pero `update` o `create-drop` en desarrollo.

---

### Paso 3: Crear la base de datos en Postgres


In [ ]:
CREATE DATABASE mi_base_de_datos;

---

## Paso 4: Ejecutar el Proyecto

1. Asegúrate de que PostgreSQL esté encendido.
2. Ejecuta la aplicación desde VSCode. 
    
    `mvn clean install`

    `mvn spring-boot:run`

3. Spring Boot se conectará a la base y:

   * Creará todas las tablas.
   * Insertará datos iniciales (si agregas `data.sql`).
   * Ejecutará el script `schema.sql` (si existe y tienes permisos).

---


## **Pruebas con Postman**

### Paso 5: Probar Endpoints CRUD

Asumiendo que has implementado controladores básicos:

#### 5.1 Crear un cliente

**POST** `http://localhost:8080/api/clientes`

```json
{
  "nombre": "Juan",
  "email": "juan@example.com"
}
```

---

#### 5.2 Crear un producto

**POST** `http://localhost:8080/api/productos`

```json
{
  "nombre": "Laptop",
  "precio": 2500.00
}
```

---

#### 5.3 Crear un pedido (con productos y cliente)

**POST** `http://localhost:8080/api/pedidos`

```json
{
  "clienteId": 1,
  "productosIds": [1],
  "fecha": "2025-05-25"
}
```

---

#### 5.4 Obtener pedidos por cliente

**GET** `http://localhost:8080/api/pedidos/cliente/1`

---

## **Verificación en DBeaver/pgAdmin**

1. Conéctate a tu base de datos `mi_base_de_datos`.
2. Ejecuta:

```sql
SELECT * FROM cliente;
SELECT * FROM pedido;
SELECT * FROM producto;
SELECT * FROM pedido_productos; -- tabla intermedia de la relación ManyToMany
```

Verifica que se hayan creado las relaciones y que los datos insertados desde Postman estén visibles.

---

## **Errores comunes a explicar**

| Error                               | Posible causa                                   | Solución                                                     |
| ----------------------------------- | ----------------------------------------------- | ------------------------------------------------------------ |
| `org.postgresql.util.PSQLException` | Base de datos no existe o credenciales erradas  | Verifica `application.properties` y la existencia de la base |
| `Could not open JDBC Connection`    | PostgreSQL no está encendido                    | Asegúrate de que el servicio está activo                     |
| Datos no aparecen en DB             | `spring.jpa.hibernate.ddl-auto` mal configurado | Usa `update` o `create` para crear tablas en desarrollo      |

---

## Evaluación práctica del bloque

| Criterio                                         | Evidencia                       |
| ------------------------------------------------ | ------------------------------- |
| Base de datos PostgreSQL conectada correctamente | Proyecto se ejecuta sin errores |
| Datos insertados y consultados con Postman       | Respuestas JSON correctas       |
| Tablas correctamente generadas                   | Verificadas en pgAdmin/DBeaver  |

---


### Buenas prácticas (según normas internacionales como DDD y Clean Architecture):

| Elemento                       | Ubicación recomendada                                        |
| ------------------------------ | ------------------------------------------------------------ |
| Entidades (`@Entity`)          | `com.empresa.app.entity` o `com.empresa.app.domain.model` |
| Repositorios (`JpaRepository`) | `com.empresa.app.repository`                               |
| Servicios de negocio           | `com.empresa.app.service`                                  |
| Controladores REST             | `com.empresa.app.controller`                              |
| DTOs (si se usan)              | `com.empresa.app.dto`                                        |



---

## Recursos y Normas Internacionales Aplicadas

| Norma / Guía                      | Aplicación en Clase                                            |
| --------------------------------- | -------------------------------------------------------------- |
| **JPA 2.2**                       | Anotaciones, relaciones y validaciones                         |
| **ISO/IEC 25010**                 | Enfoque en mantenibilidad, modularidad y usabilidad del código |
| **OWASP Secure Coding Practices** | Validación de entrada y relaciones controladas                 |
| **Bean Validation 2.0 (JSR 380)** | Validación de campos obligatorios y formatos                   |
| **Clean Architecture / DDD**      | Separación por capas (Entity, Repository, Controller, Service) |

---


## Material Complementario

* Repositorio GitHub con estructura base del proyecto.
* Diagrama UML editable.
* Cheatsheet de anotaciones JPA y validaciones.
* Enlaces a documentación oficial:

  * [Spring Data JPA](https://docs.spring.io/spring-data/jpa/docs/current/reference/html/)
  * [Bean Validation](https://beanvalidation.org/)
  * [Hibernate ORM](https://hibernate.org/orm/documentation/)

---
